In [1]:
pip install neuralintents 


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
pip install mplfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.1/73.1 kB 2.7 MB/s eta 0:00:00


In [3]:
from neuralintents import GenericAssistant

In [4]:
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader as web
import mplfinance as mpf
import pickle
import datetime as dt
import sys

In [6]:
portfolio={'AAPL':20, 'TSSLA':15, 'GS':10}
with open('portfolio.pkl','wb')as f:
  portfolio = pickle.dump(portfolio,f)

In [7]:
portfolio={'Apple':20, 'Tesla':5, 'Ibm':10}
with open('portfolio.pkl','rb')as f:
  portfolio = pickle.load(f)
print(portfolio)

{'AAPL': 20, 'TSSLA': 15, 'GS': 10}


In [8]:
def greetings():
  pass

In [9]:
def save_portfolio():
  with open('portfolio.pkl','wb')as f:
    pickle.dump(portfolio, f)

In [10]:
def add_portfolio():
  ticker = input("Which stock do you want to add: ")
  amount = input("How many shares do you want to add: ")

  if ticker in portfolio.keys():
    portfolio[ticker]+=amount
  else:
    portfolio[ticker]=amount

  save_portfolio()

In [11]:
def remove_portfolio():
  ticker = input("Which stock do you want to sell: ")
  amount = input("How many shares do you want to sell: ")

  if ticker in portfolio.keys():
    if amount<=portfolio[ticker]:
      portfolio[ticker]-=amount
      save_portfolio()
    else:
      print("You don't have enough shares")
  else:
    print("You don't own any shares of {ticker}")

In [12]:
def show_portfolio():
    print("Your portfolio: ")
    for ticker in portfolio.keys():
        print(f"YOu own {portfolio[ticker]} shares of {ticker}")

In [13]:
def portfolio_worth():
    sum = 0
    for ticker in portfolio.keys():

        print(ticker)
        data = web.get_data_yahoo(ticker)
        price = data['Close'].iloc[-1]
        sum += price
    print(f"Your portfolio is worth {sum} USD")

In [14]:
def portfolio_gains():
    starting_date = input("Enter a date of comparison (YYYY-MM-DD): ")
    sum_now=0
    sum_then=0
    try:
        for ticker in portfolio.keys():
            data = web.get_data_yahoo(ticker)
            price_now = data['Close'].iloc[-1]
            price_then = data.loc[data.index == starting_date]['Close'].values[0]
            sum_now += price_now
            sum_then += price_then

        print(f"Relative Gains: {((sum_now-sum_then)/sum_then)*100}%")
        print(f"Absolute Gains: {sum_now-sum_then} USD")
    except IndexError:
        print("There was no trading on this day!")

In [15]:
def plot_chart():
    ticker = input("Choose a ticker symbol: ")
    starting_string = input("Choose a starting date (DD/MM/YYYY): ")
    plt.style.use('dark_background')
    start = dt.datetime.strptime(starting_string, "%d/%m/%Y")
    end = dt.datetime.now()
    data=web.get_data_yahoo(ticker,start, end)
    colors = mpf.make_marketcolors(up='#00ff00', down='#ff0000', wick = ' inherit', edge ='inherit', volume = 'in')
    mpf_style = mpf.make_mpf_style(base_mpf_style='nightclouds', marketcolors=colors)
    mpf.plot(data,type='candle', style = mpf_style, volume = True)

def bye():
    print("Goodbye")
    sys.exit(0)

In [ ]:
mappings = {
    # 'greetings': greetings,
    'plot_chart':plot_chart,
    "add_portfolio":add_portfolio,
    'remove_portfolio':remove_portfolio,
    'show_portfolio':show_portfolio,
    'portfolio_worth':portfolio_worth,
    'portfolio_gains':portfolio_gains,
    'bye':bye
}

assistant = GenericAssistant('intents.json',mappings, "financial_assistant_model")
assistant.train_model()
assistant.save_model()

In [ ]:
while True:
  message = str(input(""))
  assistant.request(message)